# Create a time-of-flight lookup table for DREAM

In [ ]:
import scipp as sc
from ess.reduce import time_of_flight
from ess.reduce.nexus.types import SampleRun, Position
from ess.reduce.time_of_flight.simulation import DiskChoppers, NumberOfSimulatedNeutrons
import scippnexus as snx


from ess.dream.beamline import InstrumentConfiguration, choppers

DISPLAY_INTERMEDIATE_STEPS = False

## Select the choppers

In [ ]:
disk_choppers = choppers(InstrumentConfiguration.high_flux)
for chopper_name, chopper in disk_choppers.items():
    if DISPLAY_INTERMEDIATE_STEPS:
        display(chopper_name)
        display(chopper)

## Setting up the workflow

In [ ]:
# Initialize the workflow
wf = time_of_flight.GenericTofWorkflow(
    tof_lut_provider=time_of_flight.TofLutProvider.TOF,  # This argument decides if the workflow uses `tof` to generate the lookup table
    # RunType and MonitorType does not matter for lookup table generation.
    run_types=[],
    monitor_types=[],
)

# Set tof simulation parameters
wf[DiskChoppers[SampleRun]] = sc.DataGroup(disk_choppers)  # Mapping of chopper names to `DisckChopper` objects
wf[Position[snx.NXsource, SampleRun]] = sc.vector([0, 0, 0], unit='m')  # Position of the source, which is usually at the origin
wf[NumberOfSimulatedNeutrons] = 5_000_000  # Number of neutrons to simulate. More neutrons means better resolution of the lookup table. Feel free to increase as much as you want.

# Set lookup table parameters
wf[time_of_flight.LtotalRange] = sc.scalar(60.0, unit="m"), sc.scalar(80.0, unit="m")
wf[time_of_flight.DistanceResolution] = sc.scalar(0.1, unit="m")
wf[time_of_flight.TimeResolution] = sc.scalar(250.0, unit='us')
wf[time_of_flight.LookupTableRelativeErrorThreshold] = 0.02
wf[time_of_flight.PulsePeriod] = 1.0 / sc.scalar(14.0, unit="Hz")
wf[time_of_flight.PulseStride] = 1
wf[time_of_flight.PulseStrideOffset] = None

In [ ]:
wf.visualize(time_of_flight.TimeOfFlightLookupTable)

## Compute the table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
table.plot()

## Save to file

In [ ]:
# Save chopper metadata
# TODO: storing the choppers as a PyObject is skipped when saving to disk
table.coords['choppers'] = sc.scalar(disk_choppers)
# Write to file
table.save_hdf5('DREAM-high-flux-tof-lookup-table.h5')

## Load the file

In [ ]:
# Initialize the workflow
wf = time_of_flight.GenericTofWorkflow(
    tof_lut_provider=time_of_flight.TofLutProvider.FILE,  # This argument decides if the workflow uses a file to load the lookup table
    run_types=[],
    monitor_types=[],
)
# Set lookup table parameters
wf[time_of_flight.TimeOfFlightLookupTableFilename] = 'DREAM-high-flux-tof-lookup-table.h5'
loaded_table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
if DISPLAY_INTERMEDIATE_STEPS:
    display(wf.visualize(time_of_flight.TimeOfFlightLookupTable))
    display(loaded_table)
table.plot()
